In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy
%matplotlib inline
plt.rcParams['font.sans-serif'] = ['SimHei'] # 指定默认字体
plt.rcParams['axes.unicode_minus'] = False # 解决保存图像时负号‘-’显示为方块的问题

In [2]:
app = pd.read_csv('../数据/application_record.csv')
score = pd.read_csv('../数据/风险评分.csv')
res = pd.read_csv('../数据/response.csv')
tra = pd.read_csv('../数据/transaction.csv')
lin = pd.read_csv('../数据/link_table.csv')

D:\software\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
tra = pd.merge(tra,res,on='account_id',how='left')
# 账户号映射客户号
id_dic = lin.set_index('account_id')['ID'].to_dict()
tra['ID'] = tra['account_id'].apply(lambda x: id_dic[x])
tra.drop('account_id',axis=1,inplace=True) # 去掉account_id

In [4]:
app.head(1)

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0


##### 窗口暂时定在半年 1998-1-1 至 1998-6-30
##### 可以根据ID出现情况定制观察窗口

In [8]:
tra['date'] = tra['date'].astype('datetime64') 
data = tra[tra.date>='1998-1-1'][tra.date<='1998-6-30']
# data = tra[tra.date>='1998-7-1'][tra.date<='1998-12-31'] # 我要保存1998下半年数据

D:\software\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
data.head(2)

,trans_id,date,type,operation,k_symbol,bank,account,amount_num,balance_num,resp,ID
905083,729845,1998-07-01,debit,cash,NaN,NaN,NaN,34500.0,32968.0,1,6208926
905084,800074,1998-07-01,credit,loan,NaN,NaN,NaN,2700.0,47912.0,1,5118353


In [10]:
data.columns

Index(['trans_id', 'date', 'type', 'operation', 'k_symbol', 'bank', 'account',
       'amount_num', 'balance_num', 'resp', 'ID'],
      dtype='object')

In [11]:
data.operation = data.operation.fillna('interest') # 因为交易类型缺失的都为交易特征的利息
data.k_symbol = data.k_symbol.fillna('Unknow')

In [12]:
app_data = copy.deepcopy(app.drop_duplicates('ID')[app.ID.isin(data.ID)])

D:\software\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


### 衍生变量 

###### 表现期的交易次数

In [13]:
fre = data.groupby('ID').resp.count().reset_index()
fre = fre.rename(columns={'resp':'Xn_trades'})
app_data = pd.merge(app_data,fre,on='ID',how='left')

##### 总交易金额

In [14]:
# sum_money = data.groupby('ID').amount_num.agg(sum).reset_index()
# sum_money = sum_money.rename(columns = {'amount_num':'交易总金额'})
# app_data = pd.merge(app_data,sum_money,on='ID',how='left')

##### 每次平均交易金额

In [15]:
ave_money = data.groupby('ID').amount_num.agg(np.mean).reset_index()
ave_money = ave_money.rename(columns = {'amount_num':'Xn_Avg_amount_num'})
app_data = pd.merge(app_data,ave_money,on='ID',how='left')
app_data['Xn_Avg_amount_num'] = app_data['Xn_Avg_amount_num'].apply(lambda x: round(x,0))

##### 账户余额

In [16]:
fina_money = data.drop_duplicates('ID',keep='last').loc[:,['ID','balance_num']]
fina_money = fina_money.rename(columns = {'balance_num':'Xn_balance_num'})
app_data = pd.merge(app_data,fina_money,on='ID',how='left')

##### 账户余额的变化

In [17]:
change_money = pd.merge(data.drop_duplicates('ID','first').loc[:,['ID','balance_num']] ,data.drop_duplicates('ID','last').loc[:,['ID','balance_num']],on='ID')
change_money.fillna(0,inplace=True)
change_money['Xn_Dif_balance_num'] = change_money.iloc[:,2] - change_money.iloc[:,1]
app_data = pd.merge(app_data,change_money.loc[:,['ID','Xn_Dif_balance_num']],on='ID',how='left')

### 交易类型占比

In [18]:
# 先计算计数
cnt = pd.DataFrame()
cnt['ID'] = app_data.ID
for i in list(data.operation.value_counts().index.values):
    cnt = pd.merge(cnt,data[data.operation==i].groupby('ID').operation.count().reset_index().rename(columns={'operation':('Xr_'+i)}),on='ID',how='left')
cnt.fillna(0,inplace=True)
cnt.head()

,ID,Xr_cash,Xr_remit,Xr_interest,Xr_loan,Xr_collection,Xr_cc_debit
0,5112862,17.0,18.0,6.0,0.0,6.0,0.0
1,5009180,13.0,6.0,6.0,0.0,6.0,0.0
2,5009222,12.0,6.0,6.0,0.0,6.0,0.0
3,5009363,11.0,6.0,6.0,6.0,0.0,3.0
4,6153587,21.0,12.0,6.0,0.0,6.0,0.0


In [19]:
# 变成占比
per = copy.deepcopy(cnt)
per.set_index('ID',inplace=True)
per['sum'] = per.agg(sum,axis=1)
per.loc[:,'Xr_cash':'Xr_cc_debit'] = per.loc[:,'Xr_cash':'Xr_cc_debit'].apply(lambda x: round(x/per['sum'],2))
per.drop('sum',axis=1,inplace=True)
per.head()

,Xr_cash,Xr_remit,Xr_interest,Xr_loan,Xr_collection,Xr_cc_debit
ID,,,,,,
5112862,0.36,0.38,0.13,0.00,0.13,0.00
5009180,0.42,0.19,0.19,0.00,0.19,0.00
5009222,0.40,0.20,0.20,0.00,0.20,0.00
5009363,0.34,0.19,0.19,0.19,0.00,0.09
6153587,0.47,0.27,0.13,0.00,0.13,0.00


In [20]:
app_data = pd.merge(app_data,per,on='ID',how='left')

### 交易特征

###### 是否有房贷、养老金、保险交易、贷款交易

In [21]:
dic = {'mortgage':'Xc_mortgage','annuity':'Xc_annuity','insurance_pay':'Xc_insurance_pay','loan_repay':'Xc_loan_repay'}
for k in dic:   
    id1 = data[data.k_symbol==k].drop_duplicates('ID').ID.reset_index()
    id1[dic[k]] = 1; id1.drop(['index'],axis=1,inplace=True)
    app_data = pd.merge(app_data,id1,on='ID',how='left')

#### RFM

In [22]:
rfm = pd.read_csv('../数据/rfm_label.csv')
rfm = rfm.rename(columns={'label':'Xc_rfm'})
app_data = pd.merge(app_data,rfm.loc[:,['ID','Xc_rfm']],on='ID',how='left')

In [23]:
app_data.fillna(0,inplace=True)

### 个人信息变量

In [24]:
# 风险评分
app_data = pd.merge(app_data,score.loc[:,['ID','credit_score','score']],on='ID',how='left')

In [25]:
app_data = pd.merge(app_data,data.drop_duplicates('ID').loc[:,['ID','resp']],on='ID',how='left')
app_data.insert(0,'resp',app_data.pop('resp')) # 更改resp列的位置

In [26]:
app_data.DAYS_BIRTH = (-app.DAYS_BIRTH/365).apply(lambda x: round(x))
app_data.DAYS_EMPLOYED = (-app.DAYS_EMPLOYED/365).apply(lambda x: round(x))
app_data.loc[app_data.DAYS_EMPLOYED==-1001,'DAYS_EMPLOYED'] = 'Unknow'

In [27]:
app_data.loc[app_data.OCCUPATION_TYPE==0,'OCCUPATION_TYPE'] = 'Unknow'

In [28]:
app_data = app_data.rename(columns={'CODE_GENDER':'Xc_gender','FLAG_OWN_CAR':'Xc_car','FLAG_OWN_REALTY':'Xc_realty',
                         'CNT_CHILDREN':'Xn_children','AMT_INCOME_TOTAL':'Xn_income','credit_score':'Xn_credit_score','score':'Xn_score',
                        'NAME_INCOME_TYPE':'Xc_income_type','NAME_EDUCATION_TYPE':'Xc_edu', 'NAME_FAMILY_STATUS':'Xc_marry', 'NAME_HOUSING_TYPE':'Xc_housing',
                        'DAYS_BIRTH':'Xn_age', 'DAYS_EMPLOYED':'Xn_employed', 'FLAG_MOBIL':'Xc_phone', 'FLAG_WORK_PHONE':'Xc_work_phone',
                        'FLAG_EMAIL':'Xc_email', 'OCCUPATION_TYPE':'Xc_occupation', 'CNT_FAM_MEMBERS':'Xn_family'})

In [29]:
app_data.to_csv('../数据/营销模型建模数据.csv')

In [30]:
app_data.columns

Index(['resp', 'ID', 'Xc_gender', 'Xc_car', 'Xc_realty', 'Xn_children',
       'Xn_income', 'Xc_income_type', 'Xc_edu', 'Xc_marry', 'Xc_housing',
       'Xn_age', 'Xn_employed', 'Xc_phone', 'Xc_work_phone', 'FLAG_PHONE',
       'Xc_email', 'Xc_occupation', 'Xn_family', 'Xn_trades',
       'Xn_Avg_amount_num', 'Xn_balance_num', 'Xn_Dif_balance_num', 'Xr_cash',
       'Xr_remit', 'Xr_interest', 'Xr_loan', 'Xr_collection', 'Xr_cc_debit',
       'Xc_mortgage', 'Xc_annuity', 'Xc_insurance_pay', 'Xc_loan_repay',
       'Xc_rfm', 'Xn_credit_score', 'Xn_score'],
      dtype='object')